### DLRM Training

### Load embeddings into datasets

In [ ]:
import tensorflow as tf
from google.cloud import storage

# GCS paths
USER_EMBEDDINGS_PATH = 'gs://vector_bucket/user_embeddings.npy'
POST_EMBEDDINGS_PATH = 'gs://vector_bucket/post_embeddings.npy'
USER_TFRECORD_PATH = 'gs://user_bucket/*.tfrecord'
POST_TFRECORD_PATH = 'gs://post_bucket/*.tfrecord'

# Load embeddings from GCS
def load_embeddings(gcs_path):
    client = storage.Client()
    bucket_name, object_name = gcs_path.replace('gs://', '').split('/', 1)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(object_name)
    embedding_bytes = blob.download_as_string()
    embeddings_tensor = tf.io.parse_tensor(embedding_bytes, out_type=tf.float32)
    return embeddings_tensor

user_embeddings = load_embeddings(USER_EMBEDDINGS_PATH)
post_embeddings = load_embeddings(POST_EMBEDDINGS_PATH)

# Load additional sparse features
def load_tfrecord_data(file_pattern, feature_description):
    raw_dataset = tf.data.TFRecordDataset(tf.io.gfile.glob(file_pattern))
    return raw_dataset.map(lambda x: tf.io.parse_single_example(x, feature_description))

user_feature_description = {
    'viewed_posts': tf.io.FixedLenFeature([], tf.float32),
    'upvoted_posts': tf.io.FixedLenFeature([], tf.float32),
    'commented_posts': tf.io.FixedLenFeature([], tf.float32),
    'login_frequency': tf.io.FixedLenFeature([], tf.float32)
}

post_feature_description = {
    'post_length': tf.io.FixedLenFeature([], tf.float32),
    'vote_count': tf.io.FixedLenFeature([], tf.float32),
    'upvote_count': tf.io.FixedLenFeature([], tf.float32),
    'view_count': tf.io.FixedLenFeature([], tf.float32),
    'comment_count': tf.io.FixedLenFeature([], tf.float32),
    'avg_time_viewed': tf.io.FixedLenFeature([], tf.float32),
    'ctr': tf.io.FixedLenFeature([], tf.float32)
}

user_dataset = load_tfrecord_data(USER_TFRECORD_PATH, user_feature_description)
post_dataset = load_tfrecord_data(POST_TFRECORD_PATH, post_feature_description)

### Define and instantiate DLRM Model

In [ ]:
# Define the DLRM model class
class DLRMModel(tf.keras.Model):
    def __init__(self, embedding_dim, sparse_feature_dim):
        super().__init__()
        
        # Dense layers for sparse features
        self.user_sparse_dense = tf.keras.layers.Dense(sparse_feature_dim, activation='relu')
        self.post_sparse_dense = tf.keras.layers.Dense(sparse_feature_dim, activation='relu')
        
        # Final interaction layers
        self.interaction_layer = tf.keras.layers.Dense(128, activation='relu')
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, user_emb, post_emb, user_sparse, post_sparse):
        
        # Process sparse features
        user_sparse_vector = self.user_sparse_dense(user_sparse)
        post_sparse_vector = self.post_sparse_dense(post_sparse)
        
        # Concatenate all features
        interaction = tf.concat([user_emb, post_emb, user_sparse_vector, post_sparse_vector], axis=1)
        
        # Final interaction and prediction
        x = self.interaction_layer(interaction)
        return self.output_layer(x)

# Model instantiation
embedding_dim = 32
sparse_feature_dim = 16
dlrm_model = DLRMModel(embedding_dim, sparse_feature_dim)

In [ ]:
# Training setup
BATCH_SIZE = 256
EPOCHS = 5
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training loop
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    
    for user, post in zip(user_dataset.batch(BATCH_SIZE), post_dataset.batch(BATCH_SIZE)):
        with tf.GradientTape() as tape:
            user_emb = user_embeddings
            post_emb = post_embeddings

            user_sparse = tf.stack([user['viewed_posts'], user['upvoted_posts'], user['commented_posts'], user['login_frequency']], axis=1)
            post_sparse = tf.stack([post['post_length'], post['vote_count'], post['upvote_count'], post['view_count'], post['comment_count'], post['avg_time_viewed'], post['ctr']], axis=1)

            # Forward pass
            predictions = dlrm_model(user_emb, post_emb, user_sparse, post_sparse)

            # Use binary cross-entropy as the loss
            labels = tf.ones_like(predictions)  # Replace with actual labels if available
            loss = tf.keras.losses.binary_crossentropy(labels, predictions)

        # Backward pass
        gradients = tape.gradient(loss, dlrm_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, dlrm_model.trainable_variables))

    print(f"Loss: {tf.reduce_mean(loss).numpy()}")

print("Training complete.")

# Save the trained model for inference or further usage
dlrm_model.save("gs://model_bucket/models/dlrm_model")
